Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name below:

In [ ]:
let name = ""
let rollno = ""

## Important notes about grading:

1. **Compiler errors:** All code you submit must compile. Programs that do not compile will probably receive an automatic zero. If you are having trouble getting your assignment to compile, please visit consulting hours. If you run out of time, it is better to comment out the parts that do not compile, than hand in a more complete file that does not compile.
2. **Late assignments:** Please carefully review the course website's policy on late assignments, as all assignments handed in after the deadline will be considered late. Verify on moodle that you have submitted the correct version, before the deadline. Submitting the incorrect version before the deadline and realizing that you have done so after the deadline will be counted as a late submission.

# GADTs

Let's start with the definition of Church numerals.

In [1]:
type z = Z
type 'n s = S : 'n -> 'n s

type z = Z


type 'n s = S : 'n -> 'n s


## Cross product

A cross product of two lists is given by the function:

```ocaml
cross : 'a list -> 'b list -> ('a,'b) list
```

which returns a new list with all possible pairs of the elements of the two lists. The implementation of the function in OCaml is:

In [2]:
(* scalar product of value with a list *)
let rec cross_v_l v l =
  match l with
  | [] -> []
  | x::xs -> (v,x)::cross_v_l v xs
  
let rec cross l1 l2 = 
  match l1 with
  | [] -> []
  | x::xs -> (cross_v_l x l2) @ cross xs l2

val cross_v_l : 'a -> 'b list -> ('a * 'b) list = <fun>


val cross : 'a list -> 'b list -> ('a * 'b) list = <fun>


Here are some examples:

In [3]:
cross [1;2] ["a";"b";"c";"d"];;
cross [] [];;
cross [] [1];;
cross [1] ["a"];;

- : (int * string) list =
[(1, "a"); (1, "b"); (1, "c"); (1, "d"); (2, "a"); (2, "b"); (2, "c");
 (2, "d")]


- : ('a * 'b) list = []


- : ('a * int) list = []


- : (int * string) list = [(1, "a")]


Naturally, if `l1` is of length `n` and `l2` is of length `m`, then the resultant list is of length `n * m`. Let us enforce this property in a length-indexed list.

In [4]:
(* Length-indexed list *)
type (_,_) list =
  | Nil  : ('a, z ) list
  | Cons : 'a * ('a,'n) list -> ('a, 'n s) list

type (_, _) list =
    Nil : ('a, z) list
  | Cons : 'a * ('a, 'n) list -> ('a, 'n s) list


## Problem 1

The scalar product of a value with a list of values does not change the length of the list. Implement the function 

```ocaml
cross_v_l : 'a -> ('b, n) list -> ('a * 'b, n) list
```

In [ ]:
let cross_v_l v l = 
  (* YOUR CODE HERE *)
  raise (Failure "Not implemented")

In [ ]:
assert (cross_v_l 1 (Cons ("a", Cons ("b", Nil))) = Cons ((1, "a"), Cons ((1, "b"), Nil)))

## Appending lists 

Observe in `cross` that we have used `append` function. For reference, the append function on regular list is:

In [ ]:
let rec append l1 l2 =
  match l1 with
  | [] -> l2 
  | x::xs -> x::append xs l2

We need a corresponding implementation of append for the length-indexed case. Since the length of the result of the append of two lists is the sum of the lengths of the two lists, we need carry around proofs for the sum of length as we had seen in the lecture for tail recursive reverse of a list. In particular, we will show that:

\\[
\begin{array}{c}
0 + n = n \\
m+n = o \implies (m+1)+n = o+1
\end{array}
\\]

The two axioms above will correspond to the two cases in the body of the append.

## Problem 2

Complete the implementation of the following definition of `plus` which correspond to the theorems above

In [ ]:
(* uncomment and implement

type (_,_,_) plus =
  | PlusZero  : (z,?,?) plus
  | PlusSucc : (?, ?, 'o) plus -> (?, ?, 'o s) plus
  
*)

(* YOUR CODE HERE *)
raise (Failure "Not implemented")

In [ ]:
let zero_plus_zero_eq_zero : (z,z,z) plus = PlusZero

In [ ]:
let two_plus_three_eq_five : (z s s, z s s s, z s s s s s) plus = PlusSucc (PlusSucc PlusZero)

## Problem 3

Implement the append for length indexed lists of length `m` and `n`. The first argument is the proof that `m + n = o`.

In [ ]:
let rec append : type m n o. (m,n,o) plus -> ('a, m) list -> ('a, n) list -> ('a, o) list =
  (* YOUR CODE HERE *)
  raise (Failure "Not implemented")

In [ ]:
assert (append zero_plus_zero_eq_zero Nil Nil = Nil);
assert (append two_plus_three_eq_five (Cons (1,Cons(2,Nil))) (Cons (3, Cons(4,Cons(5,Nil)))) 
        = Cons (1,Cons(2,Cons(3,Cons(4,Cons(5,Nil))))))
        

## Multiplication

Let's consider the implementation of `cross`:

```ocaml
let rec cross l1 l2 = 
  match l1 with
  | [] -> []
  | x::xs -> (cross_v_l x l2) @ cross xs l2
```

Given a list `l1` of length `m` and `l2` of length `n`, the result will be a list of length `m * n`. We do not have multiplication on type-level church numerals. Hence, we will implement them ourselves. 

Let's consider the empty case (first case) in `cross`. If `l1` is empty, then the length of the result list is empty whatever the length of `l2` is. This corresponds to the axiom 

\\[
0 * n = 0 \quad (ax1)
\\]

Now focus on the non-empty case in `cross`. There is a call to `append` (`@`), a call to `cross_v_l` and a recursive call to `cross`. For the corresponding imlementation using length-indexed lists, `cross_v_l` does not need an explicit proof as you have implemented earlier. We know that `cross_v_l x l2` will be of length `n` if `l2` is of length `n`. But we need proofs for `append` and the recursive call of `cross`. 

Given that `l1` is a non-empty list in the non-empty case, we can represent its length as `m s`. That is to say that the length of `l1` is non-zero. Then `xs` is of length `m`. The recursive call on `cross` is now on lists of length `m` and `n`. Let us say that `m * n = p`. We know that the length of the result of `cross_v_l x l2` call is `n`. Hence, `append` is applied on lists of lengths `n` and `p`. Let us say that `n + p = o`.

Putting all the facts together:

\\[
n + p = o ~\wedge~ m * n = p \implies (m+1) * n = o \quad (ax2)
\\]

This can be shown to be true using the arithmetic axioms of distributivity and multiplicative identity. The axioms $ax1$ and $ax2$ are precisely what we need for implementing `cross` for length-indexed lists.

## Problem 4

Complete the definition of `mult` type below based on the axioms $ax1$ and $ax2$.

In [ ]:
(* UNCOMMENT AND IMPLEMENT

type (_,_,_) mult =
  | MultZero : (?, ?, ?) mult
  | MultSucc : (?, ?, ?) plus * (?, ?, ?) mult -> ('m s, 'n, 'o) mult
  
*)

(* YOUR CODE HERE *)
raise (Failure "Not implemented")

In [ ]:
let zero_mult_two_eq_zero : (z, z s s, z) mult = MultZero

In [ ]:
let one_mult_two_eq_two : (z s, z s s, z s s) mult = MultSucc (PlusSucc (PlusSucc PlusZero), MultZero)

## Problem 5

In the following, write down terms (proofs) which have those types (theorems).

In [ ]:
let two_mult_two_eq_four : (z s s, z s s, z s s s s) mult =
  (* YOUR CODE HERE *)
  raise (Failure "Not implemented")

let two_mult_one_eq_two : (z s s, z s, z s s) mult =
  (* YOUR CODE HERE *)
  raise (Failure "Not implemented")

In [ ]:
let _two_mult_two_eq_four : (z s s, z s s, z s s s s) mult = two_mult_two_eq_four in
let _two_mult_one_eq_two : (z s s, z s, z s s) mult = two_mult_one_eq_two in
()

## Problem 6

Implement cross which accepts the multiplication proof as the first argument.

In [ ]:
let rec cross : type m n o. (m,n,o) mult -> ('a,m) list -> ('b,n) list -> ('a * 'b, o) list =
  (* YOUR CODE HERE *)
  raise (Failure "Not implemented")

In [ ]:
assert (cross zero_mult_two_eq_zero Nil (Cons (1,Cons(2,Nil))) = Nil);
assert (cross one_mult_two_eq_two (Cons("a", Nil)) (Cons (1,Cons(2,Nil))) =
        Cons(("a",1), Cons(("a",2), Nil)))

# Zipping

You will now implement zip functions that will zip length indexed lists.

## Problem 7

First, implement zip function that works on the same lengthed lists. Compared to the standard library functions such as `map2` and `iter2`, which raises an `Invalid_argument` exception when the lists are of different lengths, your `zip` function will statically reject lists of different lengths. 

In [ ]:
let rec zip : type n. ('a,n) list -> ('b,n) list -> ('a *'b, n) list =
  (* YOUR CODE HERE *)
  raise (Failure "Not implemented")

In [ ]:
assert (zip (Cons(1,Cons(2,Nil))) (Cons("a",Cons("b",Nil))) = 
        Cons((1,"a"), Cons((2,"b"), Nil)))

## Zipping lists of unequal length

Let's say we want to allow zipping lists of unequal lengths such that we only zip together the first `n` elements where `n` is the smaller of the lengths of the two lists. For example,

```ocaml
let rec zip_matching l1 l2 =
  match l1,l2 with
  | [],_ -> []
  | _,[] -> []
  | x::xs,y::ys -> (x,y)::zip_matching xs ys
;;
```

```ocaml
assert (zip_matching [1;2] [1.0;2.0;3.0] = [(1,1.0);(2;2.0)])
```

The extra element in the second list are ignored. If the lists are of lengths, `m` and `n`, then the result list is of length `o` where `o` is the minimum of `m` and `n`. We do not have minimum function on type level-numerals. Hence, we will implement them. 

## Problem 8

Implement the `min` type that corresponds to the three cases in the body of the zip_matching function

In [ ]:
(* UNCOMMENT AND IMPLEMENT

type (_,_,_) min =
  | MinZero1 : ? 
  | MinZero2 : ? 
  | MinSucc  : ?
*)

(* YOUR CODE HERE *)
raise (Failure "Not implemented")

In [ ]:
let min_zero_four_zero : (z, z s s s s, z) min = MinZero1

## Problem 9

In the following, write down terms (proofs) which have those types (theorems).

In [ ]:
let min_three_zero_zero : (z s s s, z, z) min = 
  (* YOUR CODE HERE *)
  raise (Failure "Not implemented")

let min_three_five_eq_three : (z s s s, z s s s s s, z s s s) min = 
  (* YOUR CODE HERE *)
  raise (Failure "Not implemented")

In [ ]:
let _min_three_zero_zero : (z s s s, z, z) min = min_three_zero_zero

## Problem 10

Implement the `zip_matching` function on length-indexed list:

In [ ]:
let rec zip_matching : type n m o. (n,m,o) min -> ('a, n) list -> ('b, m) list -> ('a * 'b, o) list =
  (* YOUR CODE HERE *)
  raise (Failure "Not implemented")

In [ ]:
assert (zip_matching min_zero_four_zero Nil (Cons(1,(Cons(2,Cons(3,Cons(4,Nil)))))) = Nil);
assert (zip_matching min_three_zero_zero  (Cons(1,(Cons(2,Cons(3,Nil))))) Nil = Nil)